In [ ]:
# 패키지 임포트
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize, Bounds, LinearConstraint

import yfinance as yf
from arch import arch_model
import scipy

### GARCH 모델 함수 구현

In [ ]:
# GARCH(1, 1) 모델 클래스
class garchOneOne:

    # 초기화 함수
    def __init__(self, rets):

        # 수익률
        self.rets = rets * 100


    # GARCH 모델에 기반한 변동성 추정치 산출 함수
    